# Estimating crew members a ship will require using pyspark.ml

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 34.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=a55a979203add3eddad10e5c9ca0bc1b504f19940c8fc2d27a5f9075101a60ae
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [9]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [4]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [6]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema = True, header = True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [27]:
#prediction based on number of crews. Hence, create a VectorAssembler

vecAssembler = VectorAssembler(inputCols=["cabins", "Tonnage", "passengers", "length"], outputCol='features')

In [14]:
vec_df = vecAssembler.transform(df)

In [15]:
vec_df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[3.55,30.27699999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[3.55,30.27699999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[7.43,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[14.88,110.0,29.7...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[13.21,101.353,26...|
+-----------+-----------+---+------------------+--------

In [25]:
# #create a data structure that has a linear regression model

lr = LinearRegression(featuresCol='features',labelCol='crew')

In [26]:
#fit our data to it

LinRegModel = lr.fit(vec_df)

In [24]:
#Linear Regression Model is specified by four properties, the coefficients and the intercept

LinRegModel.coefficients

DenseVector([0.7908, 0.0163, -0.1499, 0.3976])

In [23]:
LinRegModel.intercept

-0.8187095907145221

In [21]:
#measure the Root Means Squared Error.

LinRegModel.summary.rootMeanSquaredError

0.9625495232659861